# Demonstration of Proof-of-Work and Mining using Python

### Importing the SHA-256 hashing algorithm

In [1]:
from hashlib import sha256

### Defining the function that will output the data using the hashing algorithm

Hashing algorithms are the backbone of the security in the blockchain. They are one-way mathematical functions that are computationally infeasible to reverse-engineer. They take in input data, and output data of a fixed length.

In [2]:
def updatehash(*args):
    hashing_text = ''; h = sha256()
    for arg in args:
        hashing_text += str(arg)

    h.update(hashing_text.encode('utf-8'))
    return h.hexdigest()


#### An elementary comparison/explanation of hashing: Name Initials

Assume that in this case, we have a list of people's names. <br>
The names will be the input data, and we want the output data to be a fixed string of 3 letters.<br><br>
```"John F. Kennedy"```  &rarr;  ```"JFK"```<br>
```"George W. Bush"```  &rarr;  ```"GWB"```<br>
```"Franklin D. Roosevelt"```  &rarr;  ```"FDR"```<br><br>
However, we know that in defining initials there is a rule where we take the first letter of the first, middle, and last names. This is an example of a one-way function; if someone were to ask you to guess the whole name only given three initials, say, GWB, you would <i>most likely</i> not be able to guess George W. Bush. <br><br>
Hashing functions work like this. However, in hashing, there is NO rule or pattern from the input to output data. It is impossible to guess the exact input given an output hash. Take a look at the next example after the function we have defined.

In [3]:
updatehash("GWB")

'1355d8b4abf4f35eeec97303c7a0fa7087d8144d8905c09cb016912f8942e9c5'

In [4]:
updatehash("GWb")

'e7593b8d063b1ac11de6979fde149d89ea6dbe01a6bc7a6e48d2ce59c75b1cf5'

The two output hashes above are completely different. Even one small change, in this case, changing a letter to lowercase, results in a completely different hash.<br><br>
Hashing works with any form of input data. Here is an example of a hash of a list of strings. Notice that regardless of the size of the input data, we will always get a 64 digit sequence comprised of the numbers ```0-9``` and letters ```a-f```. This is because the hashing algoriths follow a hexadecimal numbering system.

In [5]:
updatehash(['Hello', 'Goodbye'])

'c5fa9189ee592f7fd1d4ddd5169353ec08971ebdaead1ab0f560fe299c43433b'

### Constructing the Block Class

Using OOP, we create a new class called a ```Block```. This will allow us to define certain variables and methods that we want the ```Block``` object to have. <br>
Every block in a blockchain contains certain elements of data. Because we are implementing a very simplified blockchain, I have only included the essential elements: ```data```, ```hash```, ```nonce```, ```previous_hash```. <br>
The previous hash is the hash of the mined preceding block. Since this element is used in each block, this means that the previous block is associated with the next block. Since any change affects the output hash, this ensures that the data in the blocks that have already been mined in the chain have not been tampered with.

In [6]:
class Block():
    data = None
    hash = None
    nonce = 0
    previous_hash = '0'* 64

    def __init__(self, data, number = 0):
        self.data = data
        self.number = number

    def hash(self):
        return updatehash(
            self.previous_hash,
            self.number,
            self.data,
            self.nonce
        )

    def __str__(self):
        return str('Block #: %s\nHash: %s\nPrevious Hash: %s\nData: %s\nNonce: %s\n' %(
            self.number,
            self.hash(),
            self.previous_hash,
            self.data,
            self.nonce
        ))

### Constructing the Blockchain Class

Again using OOP, we create a new class called a ```Blockchain```. Here we have defined functions that will add, remove, and mine blocks to the chain. <br>
I have also defined a function that checks whether the block is valid (meaning that the data in the block has not been modified).

In [7]:
class Blockchain():

    def __init__(self, difficulty, chain = []):
        self.difficulty = difficulty
        self.chain = chain

    def add(self, block):
        self.chain.append(block)

    def remove(self, block):
        self.chain.remove(block)

    def mine(self, block):
        try:
            block.previous_hash = self.chain[-1].hash()
        except IndexError:
            pass
        while True:
            if block.hash()[:self.difficulty] == "0" * self.difficulty:
                self.add(block); break
            else:
                block.nonce += 1

    def isValid(self):
        for i in range(1, len(self.chain)):
            _previous = self.chain[i].previous_hash
            _current = self.chain[i-1].hash()
            if _previous != _current or _current[:self.difficulty] != '0'* self.difficulty:
                return False
        return True

### Defining the Proof-of-Work Function

This is essentially where the consensus algorithm comes in. Remember that earlier above in the ```Block``` class object there is an element called a ```nonce```. The way PoW works is that the computer is changing an element of the block (the nonce) and recomputing a hash that meets the requirements of the consensus algorithm. Every piece of data in the block remains the same, but the nonce is just a number that is used incrementally to find a hash that satisfies the consensus algorithm rule.

In [8]:
def mine(data, difficulty):
    blockchain = Blockchain(difficulty)

    num = 0
    for item in data:
        num += 1
        blockchain.mine(Block(item, num))

    for block in blockchain.chain:
        print(block)

In [9]:
blockchain1 = mine(["hi", "bye", "hello"], 2)

Block #: 1
Hash: 00721179dd91101323879a0df33f8a6a151842ad7c67303b3edfb7ef1249fa40
Previous Hash: 0000000000000000000000000000000000000000000000000000000000000000
Data: hi
Nonce: 240

Block #: 2
Hash: 0001a9af267b9ca10efb5923aa1bcb4f1e3e218022a1d6b364519882b5436e8d
Previous Hash: 00721179dd91101323879a0df33f8a6a151842ad7c67303b3edfb7ef1249fa40
Data: bye
Nonce: 29

Block #: 3
Hash: 005e33e4337c6fe1a22f9bb5551668cccaef401db9c91433a033a69fdcaa957c
Previous Hash: 0001a9af267b9ca10efb5923aa1bcb4f1e3e218022a1d6b364519882b5436e8d
Data: hello
Nonce: 82



### Understanding the Above Mining Process

First, we mined the data ```["hi", "bye", "hello]``` to the blockchain with a difficulty of 2. What's happening here is that each element of this list is being verified on the chain (AKA mining). <br><br>Notice that the first block (called a genesis block) has a default previous hash of ```0000000000000000000000000000000000000000000000000000000000000000```, which we defined above when we instantiated the Block object. <br><br>The nonce of the genesis block is 240, which means when we are mining ```"hi"``` to the chain, the computer had to try a new nonce (by increasing the nonce by 1) and computed a new hash 240 times until the output hash satisfied the difficulty requirements we set above. Verifying the second data element ```"bye"``` required 29 tries, and so forth. <br>A difficulty of ```2``` forces the computer to find a hash that starts with at least two zeroes. <br><br>
As you increase the difficulty, you will see that the nonces increase substantially as we are forcing the requirements to be stricter. Not only do the nonces increase, but the time it takes for your computer to calculate and mine the data will also increase. This is why you may have heard that Proof-of-Work is computationally expensive and takes up large amounts of resources. In a real blockchain such as Ethereum, the consensus algorithms are much more rigorous.<br><br><i>Note: As of September 15th, 2022, Ethereum uses a Proof-of-Stake consensus algorithm instead of PoW, which is less ~99% less computationally expensive.</i>

Defined in the ```Blockchain``` class is an ```isValid()``` function. If you try to change the data that is already mined in the chain, for example, if you tried to change the second data element from ```"bye"``` to ```"Bye"```, this would completely change the hash and render this block and all of the following blocks as invalid.

### Cheers! Now you have seen the basic fundamentals of how a blockchain encrypts and secures data.